In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

# Завантаження датасету
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Нормалізація даних
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# One-hot-encoding міток
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Побудова моделі
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Компіляція моделі
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Навчання моделі
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=128)

# Оцінка моделі
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Точність на тестових даних: {test_acc:.4f}")








   



C:\Users\Андрій\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 40s 79ms/step - accuracy: 0.6266 - loss: 1.0262 - val_accuracy: 0.8217 - val_loss: 0.4807
Epoch 2/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 38s 81ms/step - accuracy: 0.8089 - loss: 0.5199 - val_accuracy: 0.8467 - val_loss: 0.4118
Epoch 3/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 36s 76ms/step - accuracy: 0.8374 - loss: 0.4513 - val_accuracy: 0.8591 - val_loss: 0.3731
Epoch 4/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 36s 77ms/step - accuracy: 0.8496 - loss: 0.4115 - val_accuracy: 0.8766 - val_loss: 0.3314
Epoch 5/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 37s 79ms/step - accuracy: 0.8611 - loss: 0.3778 - val_accuracy: 0.8834 - val_loss: 0.3099
Epoch 6/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 36s 76ms/step - accuracy: 0.8726 - loss: 0.3538 - val_accuracy: 0.8914 - val_loss: 0.2976
Epoch 7/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 36s 77ms/step - accuracy: 0.8768 - loss: 0.3332 - val_accuracy: 0.8944 - val_loss: 0.2851
Epoch 8/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 36s 77ms/step - accuracy: 0.8805 - loss: 0.3305 - 

In [ ]:
Досягнута точність на тестових даних становить 91.03%, що є кращим результатом порівняно з попередньою моделлю (91.01%).
Навчальна втрата (loss) стабільно зменшується.
Втрата на валідаційних даних (val_loss) також має стійку тенденцію до зменшення, хоча на окремих епохах можливі незначні коливання.
Використання Dropout у відповідних шарах запобігло перенавчанню.




In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
import numpy as np
import math

# Завантаження та підготовка даних
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Функція для створення підвибірки даних
def create_subset(X, y, subset_percentage=0.1):
    subset_size = int(len(X) * subset_percentage)
    indices = np.random.choice(len(X), subset_size, replace=False)
    return X[indices], y[indices]

# Використання 10% даних для пришвидшення навчання
subset_percentage = 0.1
X_train, y_train = create_subset(X_train, y_train, subset_percentage)
X_test, y_test = create_subset(X_test, y_test, subset_percentage)

# Підготовка зображень (зміна розміру та конвертація в RGB)
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

X_train = tf.image.resize(X_train, [32, 32])
X_test = tf.image.resize(X_test, [32, 32])
X_train = tf.image.grayscale_to_rgb(X_train)
X_test = tf.image.grayscale_to_rgb(X_test)

# One-hot encoding міток
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Аугментація даних
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)
datagen.fit(X_train)

# Завантаження VGG16 без верхніх шарів
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Розморозка верхніх 4 шарів для донавчання
for layer in base_model.layers[:-4]:
    layer.trainable = False
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Додавання нових класифікаційних шарів
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
output = Dense(10, activation='softmax', kernel_regularizer=l2(0.001))(x)

# Створення та компіляція моделі
model_vgg16 = Model(inputs=base_model.input, outputs=output)
model_vgg16.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Визначення параметрів навчання
batch_size = 64
steps_per_epoch = math.ceil(len(X_train) / batch_size)

# Навчання моделі
history = model_vgg16.fit(
    datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_test, y_test),
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    verbose=1
)

# Оцінка моделі
test_loss, test_acc = model_vgg16.evaluate(X_test, y_test, verbose=0)
print(f"Точність на тестових даних: {test_acc:.4f}")

Epoch 1/20
94/94 [==============================] - 156s 2s/step - loss: 1.5422 - accuracy: 0.5680 - val_loss: 0.9817 - val_accuracy: 0.7570
Epoch 2/20
94/94 [==============================] - 155s 2s/step - loss: 1.0777 - accuracy: 0.7253 - val_loss: 0.8861 - val_accuracy: 0.7730
Epoch 3/20
94/94 [==============================] - 155s 2s/step - loss: 0.9626 - accuracy: 0.7518 - val_loss: 0.8687 - val_accuracy: 0.7880
Epoch 4/20
94/94 [==============================] - 153s 2s/step - loss: 0.9062 - accuracy: 0.7670 - val_loss: 0.8118 - val_accuracy: 0.7990
Epoch 5/20
94/94 [==============================] - 154s 2s/step - loss: 0.8510 - accuracy: 0.7862 - val_loss: 0.8033 - val_accuracy: 0.7920
Epoch 6/20
94/94 [==============================] - 154s 2s/step - loss: 0.8218 - accuracy: 0.7910 - val_loss: 0.7344 - val_accuracy: 0.8130
Epoch 7/20
94/94 [==============================] - 155s 2s/step - loss: 0.7898 - accuracy: 0.8035 - val_loss: 0.7768 - val_accuracy: 0.8020
Epoch 8/20
94

In [ ]:
Використали модель VGG16 як базу для класифікації зображень Fashion MNIST.
Точність на тестових даних становила 0.83, що нижче порівняно з моделлю в попередньому завданні та в першій частині цього завдання (0.91).
Нижча точність пов'язана зі складністю VGG16 для задач із простими наборами даних і обмеженою кількістю навчальних прикладів.
Попри це, модель демонструє здатність адаптуватися до нових задач